<a href="https://colab.research.google.com/github/ssv273/Neural_Univesity/blob/main/hw_31.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install patool

     |████████████████████████████████| 77 kB 4.1 MB/s 


In [2]:
import os
import pandas as pd
import numpy as np
import patoolib
from tqdm.notebook import tqdm
import unicodedata
import re
import time

import tensorflow as tf


In [3]:
!mkdir data
try:
    patoolib.extract_archive("/content/drive/MyDrive/Neural_Univesity/hw_31/статьи.rar", outdir="data/")
except Exception as e:
    print(e)

patool: Extracting /content/drive/MyDrive/Neural_Univesity/hw_31/статьи.rar ...
patool: running /usr/bin/unrar x -- /content/drive/MyDrive/Neural_Univesity/hw_31/статьи.rar
patool:     with cwd='data/'
Command `['/usr/bin/unrar', 'x', '--', '/content/drive/MyDrive/Neural_Univesity/hw_31/статьи.rar']' returned non-zero exit status 9


In [4]:
files = os.listdir('data/Мед статьи 1')

In [5]:
len(files)

1776

In [6]:
def get_data(file_name):
    df = pd.read_excel('data/Мед статьи 1/' + file_name, header=None, index_col=None)
    df = df.fillna(' ')
    text = ''
    for i in range(2,6):
        text += df.iloc[i][1] + ' '
    tmp_df = pd.DataFrame()
    tmp_df.loc[0, 'abstract'] = df.iloc[1][1]
    tmp_df['text'] = text
    return tmp_df

In [7]:
data = pd.DataFrame()
bad_files = []
for file in tqdm(files):
    try:
        data = pd.concat((data, get_data(file)))
    except:
        bad_files.append(file)

  0%|          | 0/1776 [00:00<?, ?it/s]

In [8]:
bad_files

[]

In [9]:
data

,abstract,text
0,В статье представлены данные исследования влия...,Исследование функционирования половой системы ...
0,Вальгусная деформация первого пальца стопы явл...,Вальгусная деформация первого пальца стопы явл...
0,Феномен полового диморфизма в случае заболевае...,У современного человека половой диморфизм отче...
0,Настоящее исследование направлено на уточнение...,Дегенеративные изменения позвоночника встречаю...
0,Обзор посвящен основным методам системного леч...,Современное лечение костных метастазов достато...
...,...,...
0,В данной статье представлены результаты компле...,Эхинококкоз является одним из наиболее опасных...
0,В статье представлен редкий клинический опыт н...,Венозная мальформация (ВМ) спонгиозного тела у...
0,Проведено морфометрическое изучение нейронов з...,Головной мозг крыс при ЭА отличается увеличенн...
0,В статье представлены результаты исследований ...,В структуре производственных вредностей физиче...


In [10]:
def unicode_to_ascii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')


def normalize_string(s):
    s = unicode_to_ascii(s)
    s = re.sub(r'([!.?])', r' \1', s)
    s = re.sub(r'[^a-zA-Zа-яА-Я.!?]+', r' ', s)
    s = re.sub(r'\s+', r' ', s)
    return s

In [11]:
data['normalize_text'] = data['text'].apply(normalize_string)
data['normalize_abstract_in'] = data['abstract'].apply(lambda x: '<start> ' + normalize_string(x))
data['normalize_abstract_out'] = data['abstract'].apply(lambda x: normalize_string(x)  + ' <end>')

In [12]:
data.head(2)

,abstract,text,normalize_text,normalize_abstract_in,normalize_abstract_out
0,В статье представлены данные исследования влия...,Исследование функционирования половой системы ...,Исследование функционирования половои системы ...,<start> В статье представлены данные исследова...,В статье представлены данные исследования влия...
0,Вальгусная деформация первого пальца стопы явл...,Вальгусная деформация первого пальца стопы явл...,Вальгусная деформация первого пальца стопы явл...,<start> Вальгусная деформация первого пальца с...,Вальгусная деформация первого пальца стопы явл...


In [13]:
text_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
text_tokenizer.fit_on_texts(data['normalize_text'])
data_text = text_tokenizer.texts_to_sequences(data['normalize_text'])
data_text = tf.keras.preprocessing.sequence.pad_sequences(data_text,
                                                        padding='post')

In [14]:
data_text.shape

(1776, 4928)

In [15]:
abstract_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
abstract_tokenizer.fit_on_texts(data['normalize_abstract_in'])
abstract_tokenizer.fit_on_texts(data['normalize_abstract_out'])
data_abstract_in = abstract_tokenizer.texts_to_sequences(data['normalize_abstract_in'])
data_abstract_in = tf.keras.preprocessing.sequence.pad_sequences(data_abstract_in,
                                                           padding='post')

data_abstract_out = abstract_tokenizer.texts_to_sequences(data['normalize_abstract_out'])
data_abstract_out = tf.keras.preprocessing.sequence.pad_sequences(data_abstract_out,
                                                            padding='post')

In [16]:
data_abstract_in.shape, data_abstract_out.shape

((1776, 989), (1776, 989))

In [17]:
BATCH_SIZE = 5
dataset = tf.data.Dataset.from_tensor_slices(
    (data_text, data_abstract_in, data_abstract_out))
dataset = dataset.shuffle(20).batch(BATCH_SIZE)

In [18]:
def positional_embedding(pos, hidden_dim):
    PE = np.zeros((1, hidden_dim))
    for i in range(hidden_dim):
        if i % 2 == 0:
            PE[:, i] = np.sin(pos / 10000 ** (i / hidden_dim))
        else:
            PE[:, i] = np.cos(pos / 10000 ** ((i - 1) / hidden_dim))
    return PE

max_length = max(len(data_text[0]), len(data_abstract_in[0]))
HIDDEN_DIM = 128

pes = []
for i in range(max_length):
    pes.append(positional_embedding(i, HIDDEN_DIM))

pes = np.concatenate(pes, axis=0)
pes = tf.constant(pes, dtype=tf.float32)

In [19]:
pes

<tf.Tensor: shape=(4928, 128), dtype=float32, numpy=
array([[ 0.0000000e+00,  1.0000000e+00,  0.0000000e+00, ...,
         1.0000000e+00,  0.0000000e+00,  1.0000000e+00],
       [ 8.4147096e-01,  5.4030228e-01,  7.6172042e-01, ...,
         1.0000000e+00,  1.1547820e-04,  1.0000000e+00],
       [ 9.0929741e-01, -4.1614684e-01,  9.8704624e-01, ...,
         9.9999994e-01,  2.3095640e-04,  1.0000000e+00],
       ...,
       [-8.5068172e-01,  5.2568102e-01, -9.8686385e-01, ...,
         7.9197502e-01,  5.3856254e-01,  8.4258556e-01],
       [-1.7279970e-02,  9.9985069e-01, -5.1633579e-01, ...,
         7.9189360e-01,  5.3865981e-01,  8.4252334e-01],
       [ 8.3200896e-01,  5.5476224e-01,  3.1778988e-01, ...,
         7.9181212e-01,  5.3875709e-01,  8.4246117e-01]], dtype=float32)>

In [20]:
class MultiHeadAttention(tf.keras.Model):
    def __init__(self, hidden_dim, h):
        super(MultiHeadAttention, self).__init__()
        self.query_size = hidden_dim // h
        self.key_size = hidden_dim // h
        self.value_size = hidden_dim // h
        self.h = h
        self.wq = [tf.keras.layers.Dense(self.query_size) for _ in range(h)]
        self.wk = [tf.keras.layers.Dense(self.key_size) for _ in range(h)]
        self.wv = [tf.keras.layers.Dense(self.value_size) for _ in range(h)]
        self.wo = tf.keras.layers.Dense(hidden_dim)

    def call(self, query, value, mask=None):
        heads = []
        for i in range(self.h):
            score = tf.matmul(self.wq[i](query), self.wk[i](value), transpose_b=True)

            score /= tf.math.sqrt(tf.dtypes.cast(self.key_size, tf.float32))
            # (batch, query_len, value_len)
            
            # Применяем маскирование
            if mask is not None:
                score *= mask
                
                #При маскировке присваиваем ненужным значениям -1e9
                #чтобы softmax был равен 0
                score = tf.where(tf.equal(score, 0), tf.ones_like(score) * -1e9, score)

            a = tf.nn.softmax(score, axis=2)

            head = tf.matmul(a, self.wv[i](value))
            heads.append(head)

        # Соединяем все "головы внимания"
        heads = tf.concat(heads, axis=2)
        heads = self.wo(heads)
        return heads

In [21]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, hidden_dim, num_layers, h):
        super(Encoder, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.h = h
        self.embedding = tf.keras.layers.Embedding(vocab_size, hidden_dim)
        self.attention = [MultiHeadAttention(hidden_dim, h) for _ in range(num_layers)]

        self.attention_norm = [tf.keras.layers.BatchNormalization() for _ in range(num_layers)]

        self.dense_1 = [tf.keras.layers.Dense(512, activation='relu') for _ in range(num_layers)]
        self.dense_2 = [tf.keras.layers.Dense(hidden_dim) for _ in range(num_layers)]
        self.ffn_norm = [tf.keras.layers.BatchNormalization() for _ in range(num_layers)]

    def call(self, sequence, padding_mask=None):
        embed_out = self.embedding(sequence)
        embed_out += pes[:sequence.shape[1], :]
        
        sub_in = embed_out
        
        for i in range(self.num_layers):
            sub_out = self.attention[i](sub_in, sub_in, padding_mask)
            sub_out = sub_in + sub_out
            sub_out = self.attention_norm[i](sub_out)
            
            ffn_in = sub_out

            ffn_out = self.dense_2[i](self.dense_1[i](ffn_in))
            ffn_out = ffn_in + ffn_out
            ffn_out = self.ffn_norm[i](ffn_out)

            sub_in = ffn_out
            
        return ffn_out

In [22]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, hidden_dim, num_layers, h):
        super(Decoder, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.h = h
        self.embedding = tf.keras.layers.Embedding(vocab_size, hidden_dim)
        self.attention_bot = [MultiHeadAttention(hidden_dim, h) for _ in range(num_layers)]
        self.attention_bot_norm = [tf.keras.layers.BatchNormalization() for _ in range(num_layers)]
        self.attention_mid = [MultiHeadAttention(hidden_dim, h) for _ in range(num_layers)]
        self.attention_mid_norm = [tf.keras.layers.BatchNormalization() for _ in range(num_layers)]
        
        self.dense_1 = [tf.keras.layers.Dense(512, activation='relu') for _ in range(num_layers)]
        self.dense_2 = [tf.keras.layers.Dense(hidden_dim) for _ in range(num_layers)]
        self.ffn_norm = [tf.keras.layers.BatchNormalization() for _ in range(num_layers)]
        
        self.dense = tf.keras.layers.Dense(vocab_size)
        
    def call(self, sequence, encoder_output, padding_mask=None):
        # Эмбеддинги
        embed_out = self.embedding(sequence)
        embed_out += pes[:sequence.shape[1], :]
        
        bot_sub_in = embed_out
        
        for i in range(self.num_layers):
            # Нижний блок внимания
            seq_len = bot_sub_in.shape[1]
            look_left_only_mask = tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
            bot_sub_out = self.attention_bot[i](bot_sub_in, bot_sub_in, look_left_only_mask)
            bot_sub_out = bot_sub_in + bot_sub_out
            bot_sub_out = self.attention_bot_norm[i](bot_sub_out)
            
            # Верхний блок внимания
            mid_sub_in = bot_sub_out

            mid_sub_out = self.attention_mid[i](mid_sub_in, encoder_output, padding_mask)
            mid_sub_out = mid_sub_out + mid_sub_in
            mid_sub_out = self.attention_mid_norm[i](mid_sub_out)

            # Полносвязный слой
            ffn_in = mid_sub_out

            ffn_out = self.dense_2[i](self.dense_1[i](ffn_in))
            ffn_out = ffn_out + ffn_in
            ffn_out = self.ffn_norm[i](ffn_out)

            bot_sub_in = ffn_out
        
        logits = self.dense(ffn_out)
            
        return logits

In [23]:
@tf.function
def train_step(source_seq, target_seq_in, target_seq_out):
    with tf.GradientTape() as tape:
        padding_mask = 1 - tf.cast(tf.equal(source_seq, 0), dtype=tf.float32)
        padding_mask = tf.expand_dims(padding_mask, axis=1)
        encoder_output = encoder(source_seq, padding_mask)
        
        decoder_output = decoder(target_seq_in, encoder_output, padding_mask)

        loss = loss_func(target_seq_out, decoder_output)

    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))

    return loss

In [24]:
optimizer = tf.keras.optimizers.Adam()

In [25]:
crossentropy = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True)
def loss_func(targets, logits):
    mask = tf.math.logical_not(tf.math.equal(targets, 0))
    mask = tf.cast(mask, dtype=tf.int64)
    loss = crossentropy(targets, logits, sample_weight=mask)

    return loss

In [27]:
text_vocab_size = len(text_tokenizer.word_index) + 1
abstract_vocab_size = len(abstract_tokenizer.word_index) + 1

H = 8

NUM_LAYERS = 1

encoder = Encoder(text_vocab_size, HIDDEN_DIM, NUM_LAYERS, H)
decoder = Decoder(abstract_vocab_size, HIDDEN_DIM, NUM_LAYERS, H)

NUM_EPOCHS = 100

In [28]:
start_time = time.time()
for e in range(NUM_EPOCHS):
    for batch, (source_seq, target_seq_in, target_seq_out) in enumerate(dataset.take(-1)):
        loss = train_step(source_seq, target_seq_in,
                          target_seq_out)

    print('Epoch {} Loss {:.4f}'.format(
          e + 1, loss.numpy()))

    if (e + 1) % 10 == 0:
        end_time = time.time()
        print('Среднее время: {:.2f}s'.format((end_time - start_time) / (e + 1)))
        try:
            predict()
        except Exception as e:
            print(e)
            continue

Epoch 1 Loss 0.5156
Epoch 2 Loss 2.3903
Epoch 3 Loss 0.6032
Epoch 4 Loss 1.6662
Epoch 5 Loss 1.5259
Epoch 6 Loss 0.5158
Epoch 7 Loss 0.7707
Epoch 8 Loss 0.3683
Epoch 9 Loss 0.3130
Epoch 10 Loss 0.8290
Среднее время: 184.95s
name 'raw_data_en' is not defined
Epoch 11 Loss 0.1739
Epoch 12 Loss 0.2300
Epoch 13 Loss 0.3143
Epoch 14 Loss 0.3953
Epoch 15 Loss 0.3796
Epoch 16 Loss 0.0468
Epoch 17 Loss 0.0725
Epoch 18 Loss 0.1858
Epoch 19 Loss 0.0312
Epoch 20 Loss 0.0763
Среднее время: 184.24s
name 'raw_data_en' is not defined
Epoch 21 Loss 0.0438
Epoch 22 Loss 0.0946
Epoch 23 Loss 0.0193
Epoch 24 Loss 0.1777
Epoch 25 Loss 0.0127
Epoch 26 Loss 0.0474
Epoch 27 Loss 0.0191
Epoch 28 Loss 0.0506
Epoch 29 Loss 0.0977
Epoch 30 Loss 0.0085
Среднее время: 184.01s
name 'raw_data_en' is not defined
Epoch 31 Loss 0.0656
Epoch 32 Loss 0.0164
Epoch 33 Loss 0.0067
Epoch 34 Loss 0.0465
Epoch 35 Loss 0.0218
Epoch 36 Loss 0.0244
Epoch 37 Loss 0.0040
Epoch 38 Loss 0.0204
Epoch 39 Loss 0.0158
Epoch 40 Loss 0.030

In [29]:
def predict(test_source_text):

    print(test_source_text)
    test_source_seq = text_tokenizer.texts_to_sequences([test_source_text])
    print(test_source_seq)

    en_output = encoder(tf.constant(test_source_seq))

    de_input = tf.constant([[abstract_tokenizer.word_index['<start>']]], dtype=tf.int64)

    out_words = []

    while True:
        de_output = decoder(de_input, en_output)
        new_word = tf.expand_dims(tf.argmax(de_output, -1)[:, -1], axis=1)
        out_words.append(abstract_tokenizer.index_word[new_word.numpy()[0][0]])

        de_input = tf.concat((de_input, new_word), axis=-1)

        if out_words[-1] == '<end>' or len(out_words) >= 140:
            break

    print(' '.join(out_words))

In [30]:
predict('lkjsadvkjasd')

lkjsadvkjasd
[[]]
цель . изучение особенностеи течения и степени нхл в сравнении с болезнью с болезнью паркинсона уаита у пациентов с болезнью сердца . материалы и методы в исследование вошли пациентов с болезнью вен госпитализированных в возрасте лет . основную группу составили в возрасте от до лет . пациенты лечились нефрогенная у пациентов с рекомендациями . из пациентов с применением тои или эякуляция прошли мужчины и группы я группа . результаты . анализ бактериологических исследовании больных с болезнью с использованием целеи обследование пациентов с острыми соответствующими рецепторами . результаты . анализ позволило феназепамом проводилась комплексная бевацизумаб оригинальныи находились в сравнении с этим проспективное соответствии с рекомендациями проводили ремиссия причинои регионального сосудистого сопротивления индексы термическои травмы . анализ эффективности и группу составили пациентов с изолированнои форме . пациенты разделены на подгруппы я группа . результаты . резуль

In [31]:
files_2 = os.listdir('data/Мед статьи 2')

In [32]:
def get_data_2(file_name):
    df = pd.read_excel('data/Мед статьи 2/' + file_name, header=None, index_col=None)
    df = df.fillna(' ')
    text = ''
    for i in range(2,6):
        text += df.iloc[i][1] + ' '
    tmp_df = pd.DataFrame()
    tmp_df.loc[0, 'abstract'] = df.iloc[1][1]
    tmp_df['text'] = text
    return tmp_df

In [36]:
get_data_2(files_2[0])['text'].values[0]

'Причиной большинства неудовлетворительных результатов и летальных исходов хирургического лечения туберкулеза торакальные хирурги во всем мире считают бронхоплевральные осложнения [1 - 5]. Альвеолярные фистулы, возникшие при мобилизации и резекции лѐгкого во время операции, в послеоперационном периоде, становятся причиной замедленного расправления лѐгкого, в результате чего часто формируется остаточная полость. В свою очередь наличие остаточной полости ряд авторов считают решающим фактором развития эмпиемы плевры [6, 7]. При хирургическом лечении туберкулезной эмпиемы плевры могут выполняться разные виды операций: торакомиопластики, плеврэктомии с декортикацией легкого, резекционные вмешательства в сочетании с плеврэктомией и другие [8, 9, 10]. Важнейшим вопросом в выборе тактики лечения остаточной плевральной полости является наличие и размер бронхиального свища. При этом основная роль в выборе тактики принадлежит рентгенологическим методам [1]. Лечение бронхоплевральных осложнений тр

In [37]:
predict(get_data_2(files_2[0])['text'].values[0])

Причиной большинства неудовлетворительных результатов и летальных исходов хирургического лечения туберкулеза торакальные хирурги во всем мире считают бронхоплевральные осложнения [1 - 5]. Альвеолярные фистулы, возникшие при мобилизации и резекции лѐгкого во время операции, в послеоперационном периоде, становятся причиной замедленного расправления лѐгкого, в результате чего часто формируется остаточная полость. В свою очередь наличие остаточной полости ряд авторов считают решающим фактором развития эмпиемы плевры [6, 7]. При хирургическом лечении туберкулезной эмпиемы плевры могут выполняться разные виды операций: торакомиопластики, плеврэктомии с декортикацией легкого, резекционные вмешательства в сочетании с плеврэктомией и другие [8, 9, 10]. Важнейшим вопросом в выборе тактики лечения остаточной плевральной полости является наличие и размер бронхиального свища. При этом основная роль в выборе тактики принадлежит рентгенологическим методам [1]. Лечение бронхоплевральных осложнений тре

In [38]:
get_data_2(files_2[0])['abstract'].values[0]

'Цель исследования. Причиной большинства неудовлетворительных результатов и летальных исходов хирургического лечения туберкулеза торакальные хирурги во всем мире считают бронхоплевральные осложнения. Лечение бронхоплевральных осложнений требует сложного, часто многоэтапного хирургического лечения, при этом основная роль в выборе тактики принадлежит рентгенологическим методам. Результаты. Авторы приводят вариант хирургической тактики, основанной на дооперационной рентгенологической диагностике, в случае длительно существующей пострезекционной туберкулезной эмпиемы плевры с бронхоплевральным свищем. После выполнения торакомиопластики послеоперационных осложнений отмечено не было. Обсуждение. Лечение пострезекционной туберкулезной эмпиемы плевры, особенно в сочетании с бронхиальным свищем, является сложной проблемой в торакальной хирургии. Существует большое количество хирургических способов ликвидации пострезекционной полости и закрытия бронхиального свища, которые отличаются высокой тра

In [41]:
predict(get_data_2(files_2[2])['text'].values[0])

Гендерная кардиология – относительно новое направление в кардиологии и медицине, которое изучает особенности течения и лечения заболеваний в зависимости от пола, то есть различия их у женщин и мужчин. Об актуальности изучения гендерных аспектов кардиологии свидетельствует начатая в 2004 г. Американским обществом кардиологов программа «Акцент – на женщин» (Red in Women) и начатая в 2005 г. особая программа Европейского кардиологического общества «Женщины в сердце» (Women at Heart). Эти инициативы акцентируют внимание медицинских работников на росте уровня сердечно-сосудистых патологий у женщин и способствуют разработке индивидуализированных (отдельно для мужчин и женщин) подходов к выявлению и лечению этих заболеваний. Заболеваемость и смертность от сердечно-сосудистых заболеваний (ССЗ) в России остаётся на высоком уровне как среди мужчин, так и среди женщин [4]. В Европе CCЗ являются причиной смерти у 43% мужчин и 54% женщин [11]. Хотя ишемическая болезнь сердца (ИБС) и инфаркт миокард

In [42]:
get_data_2(files_2[2])['abstract'].values[0]

'В настоящее время в медицине сделан акцент на персонализированный подход, в рамках которого рассматривается учёт гендерных различий течения патологии и её ведения. В кардиологии доказательная база основана на исследованиях, включающих в большинстве случаев мужчин. Известно, что эстрогенный фон женщин обусловливает факт более позднего развития у них ИБС. Инфаркт миокарда у женщин чаще имеет атипичные проявления, что затрудняет своевременную диагностику. Несмотря на прогресс в лечении пациентов с острым коронарным синдромом, реваскуляризация у женщин проводится гораздо реже. Ввиду необходимости учёта гендерных различий для оптимизации ведения острого коронарного синдрома, инфаркта миокарда проведён анализ 84 историй болезни пациентов с инфарктом миокарда в исходе ОКСпST (34 женщины и 50 мужчин). Статистическая обработка данных проведена с использованием программы Statistica v. 10.0. По полученным данным женщины переносят инфаркт миокарда в более старшем возрасте, отличаются большей част

In [43]:
predict(get_data(files[2])['text'].values[0])

У современного человека половой диморфизм отчетливо проявляется в морфологических и физиологических показателях, частоте патологии, включая злокачественные новообразования (ЗНО). Феномен полового диморфизма в патологии, в случае заболеваемости раком желудка (РЖ), отчетливо выражен в большинстве этнических и географических популяций. Ежегодно в мире регистрируется около 1 миллиона новых случаев рака желудка, число заболевших мужчин почти в два раза больше, чем женщин [1, 2]. С наибольшей частотой РЖ встречается в ЮгоВосточной Азии (24 на 100 тыс. мужчин и 9,8 на 100 тыс. женщин), а с наименьшей – в Северной Америке (2,8 на 100 тыс. мужчин и 1,5 на 100 тыс. женщин). Во всех странах по заболеваемости лидируют мужчины: в Японии 114,7 на 100 тыс. мужчин; в Китае 46,5 на 100 тыс. мужчин. Минимальная заболеваемость наблюдается среди белых женщин в США (3,1 на 100 тыс. женщин) [3]. Рак желудка рассматривается как гетерогенное заболевание с разнообразными молекулярными и гистологическими подтип

In [44]:
get_data(files[2])['abstract'].values[0]

'Феномен полового диморфизма в случае заболеваемости раком желудка отчетливо выражен в большинстве этнических и географических популяций. Ежегодно в мире регистрируется около 1 миллиона новых случаев рака желудка, число заболевших мужчин почти в два раза больше, чем женщин. Существенные различия в частоте патологии детерминированы особенностями пищевого поведения и половыми гормонами, которые по-разному модулируют иммунный ответ у мужчин и женщин. Взаимосвязь прогестерона Р4, эстрадиола Е2, тестостерона Т4 с соответствующими рецепторами приводит к запуску механизма активации генной экспрессии цитокинов, ростовых факторов и белков, активирующих макрофаги, нейтрофилы, Т- и В-лимфоциты. Если эстрадиол активирует, то прогестерон и тестостерон относятся к факторам, ингибирующим иммунный ответ на ранних стадиях инфекции. Инфекционные заболевания, в том числе, ассоциированные с Helicobacter pylori, тяжелее протекают у мужчин, чем у женщин. Желудочный канцерогенез, как многофакторный и длитель

In [45]:
get_data(files[2])

,abstract,text
0,Феномен полового диморфизма в случае заболевае...,У современного человека половой диморфизм отче...
